In [1]:

# %%
from transformers import AutoTokenizer
from vllm import SamplingParams
import string
from vllm import LLM
from tqdm import tqdm
import random

import os


model_name="microsoft/Phi-3-medium-128k-instruct"

random.seed(42)

n_batch = 100
job_id=7

os.environ["CUDA_VISIBLE_DEVICES"] = f"{job_id}"

/home/hatakeyama/miniconda3/envs/llmeval/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-05-24 18:52:35,804	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
from datasets import load_dataset


In [3]:


# %%
print("init models...")
llm = LLM(model=model_name, trust_remote_code=True)

# %%
#tokenizer = AutoTokenizer.from_pretrained(model_name)

init models...


/home/hatakeyama/miniconda3/envs/llmeval/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


INFO 05-24 18:52:37 llm_engine.py:100] Initializing an LLM engine (v0.4.2) with config: model='microsoft/Phi-3-medium-128k-instruct', speculative_config=None, tokenizer='microsoft/Phi-3-medium-128k-instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=131072, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=microsoft/Phi-3-medium-128k-instruct)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 05-24 18:52:38 utils.py:660] Found nccl from library /home/hatakeyama/.config/vllm/nccl/cu12/libnccl.so.2.18.1
INFO 05-24 18:52:39 selector.py:27] Using FlashAttention-2 backend.
INFO 05-24 18:52:43 weight_utils.py:199] Using model weights format ['*.safetensors']
INFO 05-24 18:52:49 model_runner.py:175] Loading model weights took 26.1473 GB
INFO 05-24 18:53:04 gpu_executor.py:114] # GPU blocks: 7020, # CPU blocks: 1310


ValueError: The model's max seq len (131072) is larger than the maximum number of tokens that can be stored in KV cache (112320). Try increasing `gpu_memory_utilization` or decreasing `max_model_len` when initializing the engine.

In [32]:
max_tokens=2048*2

In [18]:
ds=load_dataset("cyberagent/chatbot-arena-ja-calm2-7b-chat-experimental", split="train")

records=ds.to_list()

# プロンプトテンプレートの準備
random.seed(42)
results=[]

cnt = 0
for i in tqdm(range(int(len(records)/n_batch))):
    # プロンプトの準備
    sampled_records = records[cnt*n_batch:(cnt+1)*n_batch]

    prompts = []
    for record in sampled_records:
        #q=record["prompt"]
        #q=q[:q.rfind(answer_template)+len(answer_template)]
        #prompts.append(tokenizer.encode(q)[:-1])
        prompts.append(record["prompt"])

    # 推論の実行
    outputs = llm.generate(
        prompts,
        #prompt_token_ids=prompts,
        sampling_params=SamplingParams(
            temperature=0.1,
            max_tokens=max_tokens,
        )
    )
    for i, output in enumerate(outputs):
        sampled_records[i]["model_answer"] = output.outputs[0].text
    results.extend(sampled_records)
    cnt += 1


# %%
len(prompts),len(sampled_records),len(outputs)

# %%
outputs

# %%



  3%|▎         | 8/292 [01:12<42:56,  9.07s/it]


KeyboardInterrupt: 

In [28]:
import datasets
new_dataset=datasets.Dataset.from_list(results)

In [31]:
new_dataset.push_to_hub("hatakeyama-llm-team/phi_chatbot_arena")

Uploading the dataset shards: 100%|██████████| 1/1 [00:02<00:00,  2.20s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/hatakeyama-llm-team/phi_chatbot_arena/commit/b0fcbafde72c44c369f5c4dde642a3e29d67ded1', commit_message='Upload dataset', commit_description='', oid='b0fcbafde72c44c369f5c4dde642a3e29d67ded1', pr_url=None, pr_revision=None, pr_num=None)

In [33]:
ds=load_dataset("llm-jp/hh-rlhf-12k-ja", split="train")

records=ds.to_list()

# プロンプトテンプレートの準備
random.seed(42)
results=[]



Generating train split: 100%|██████████| 12000/12000 [00:00<00:00, 109852.45 examples/s]


In [36]:

records=ds.to_list()

# プロンプトテンプレートの準備
random.seed(42)
results=[]

cnt = 0
for i in tqdm(range(int(len(records)/n_batch))):
    # プロンプトの準備
    sampled_records = records[cnt*n_batch:(cnt+1)*n_batch]

    prompts = []
    for record in sampled_records:
        q=records[0]["conversations"][0]["value"]
        prompts.append(q)

    # 推論の実行
    outputs = llm.generate(
        prompts,
        #prompt_token_ids=prompts,
        sampling_params=SamplingParams(
            temperature=0.1,
            max_tokens=max_tokens,
        )
    )
    for i, output in enumerate(outputs):
        sampled_records[i]["model_answer"] = output.outputs[0].text
    results.extend(sampled_records)
    cnt += 1


# %%
len(prompts),len(sampled_records),len(outputs)

# %%
outputs

# %%

new_dataset=datasets.Dataset.from_list(results)
new_dataset.push_to_hub("hatakeyama-llm-team/phi_rlhf12kja")

'なんで負け犬なんだ'